In [1]:
# =========================================================
# Enhanced Learning Pattern Analysis & Evaluation System
# =========================================================

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import (
    silhouette_score,
    davies_bouldin_score,
    calinski_harabasz_score,
    confusion_matrix
)

# ---------------------------------------------------------
# 1. LOAD & EXPLORE DATA
# ---------------------------------------------------------
df = pd.read_csv("student_dataset.csv",encoding='utf-8')

# Handle missing values
df = df.dropna(subset=['G1', 'G2', 'G3'])

# Validate data ranges
df = df[
    (df['G1'] >= 0) & (df['G1'] <= 20) &
    (df['G2'] >= 0) & (df['G2'] <= 20) &
    (df['G3'] >= 0) & (df['G3'] <= 20)
]

print(f"Dataset shape: {df.shape}")
print(f"Missing values:\n{df.isnull().sum()}\n")

# ---------------------------------------------------------
# 2. ROBUST FEATURE ENGINEERING
# ---------------------------------------------------------

# Academic Performance Metrics
df["avg_grade"] = df[["G1", "G2", "G3"]].mean(axis=1)
df["grade_variance"] = df[["G1", "G2", "G3"]].var(axis=1)
df["grade_trend"] = df["G3"] - df["G1"]
df["grade_stability"] = 1 / (1 + df["grade_variance"])  # Inverse of variance

# Study Habits & Discipline (normalized)
study_discipline_components = [
    (df["studytime"] / 4) * 0.35,  # Normalize to 0-1
    (1 - df["traveltime"] / 4) * 0.25,  # Less travel time = more discipline
    (1 - df["absences"] / df["absences"].max()) * 0.40  # Lower absences = better
]
df["study_discipline"] = sum(study_discipline_components)

# Engagement & Participation (improved mapping)
engagement_factors = {
    "activities": (df["activities"].map({"yes": 1, "no": 0})) * 0.25,
    "higher_ed": (df["higher"].map({"yes": 1, "no": 0})) * 0.20,
    "school_support": (df["schoolsup"].map({"yes": 1, "no": 0})) * 0.20,
    "internet": (df["internet"].map({"yes": 1, "no": 0})) * 0.20,
    "paid_classes": (df["paid"].map({"yes": 1, "no": 0})) * 0.15
}
df["engagement"] = sum(engagement_factors.values())

# Family Support & Environment
df["parent_education"] = (df["Medu"] + df["Fedu"]) / 8  # Normalize
df["family_support"] = (
    (df["parent_education"] * 0.35) +
    (df["famsup"].map({"yes": 1, "no": 0}) * 0.30) +
    (df["famrel"] / 5 * 0.35)  # Normalize to 0-1
)

# Socioeconomic Status
df["low_sES"] = ((df["Pstatus"].map({"T": 0, "A": 1})) * 0.5 +
                 (1 - df["Medu"] / 4) * 0.25 +
                 (1 - df["Fedu"] / 4) * 0.25)

# Lifestyle Risk Factors (normalized)
df["lifestyle_risk"] = (
    (df["goout"] / 5) * 0.30 +
    (df["Dalc"] / 5) * 0.35 +
    (df["Walc"] / 5) * 0.35
)

# Health & Wellness
df["health_score"] = 1 - (df["health"] / 5)  # Inverted (lower health = higher risk)

# ---------------------------------------------------------
# 3. PREPARE FEATURES FOR CLUSTERING
# ---------------------------------------------------------
feature_list = [
    "avg_grade",
    "grade_stability",
    "grade_trend",
    "study_discipline",
    "engagement",
    "family_support",
    "lifestyle_risk",
    "health_score",
    "low_sES"
]

# Standardize features (better than MinMaxScaler for clustering)
scaler = StandardScaler()
X = scaler.fit_transform(df[feature_list])

# ---------------------------------------------------------
# 4. OPTIMAL CLUSTERING (ELBOW + SILHOUETTE)
# ---------------------------------------------------------
silhouette_scores = []
inertias = []
k_range = range(2, 8)

for k in k_range:
    kmeans_temp = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels_temp = kmeans_temp.fit_predict(X)
    silhouette_scores.append(silhouette_score(X, labels_temp))
    inertias.append(kmeans_temp.inertia_)

optimal_k = k_range[np.argmax(silhouette_scores)]
print(f"Optimal clusters: {optimal_k} (Silhouette: {max(silhouette_scores):.3f})\n")

# ---------------------------------------------------------
# 5. FINAL CLUSTERING MODEL
# ---------------------------------------------------------
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df["cluster"] = kmeans.fit_predict(X)

# Adaptive persona labeling based on cluster characteristics
persona_mapping = {}
for cluster_id in range(optimal_k):
    cluster_data = df[df["cluster"] == cluster_id]

    avg_grade = cluster_data["avg_grade"].mean()
    grade_std = cluster_data["avg_grade"].std()
    avg_engagement = cluster_data["engagement"].mean()
    avg_discipline = cluster_data["study_discipline"].mean()
    avg_risk = cluster_data["lifestyle_risk"].mean()
    avg_absence = cluster_data["absences"].mean()
    avg_family = cluster_data["family_support"].mean()

    # More granular persona assignment
    if avg_grade >= 15:
        if avg_discipline >= 0.6:
            persona = "🌟 High Achiever - Consistent Excellence"
        else:
            persona = "💎 High Potential - Needs Discipline"
    elif avg_grade >= 12:
        if avg_discipline >= 0.5 and avg_engagement >= 0.5:
            persona = "✅ Solid Performer - On Track"
        elif avg_absence >= 15:
            persona = "⚠️ Good Grades - Attendance Issues"
        else:
            persona = "📈 Above Average - Stable"
    elif avg_grade >= 10:
        if avg_engagement >= 0.5:
            persona = "🔄 Developing Learner - Engaged"
        elif avg_discipline >= 0.5:
            persona = "💪 Struggling but Trying"
        else:
            persona = "⚠️ Low Engagement - Support Needed"
    else:  # avg_grade < 10
        if avg_absence >= 20:
            persona = "🔴 At-Risk - High Absenteeism"
        elif avg_engagement < 0.3:
            persona = "🔴 At-Risk - Disengaged"
        elif avg_risk > 0.6:
            persona = "🔴 At-Risk - Lifestyle Issues"
        else:
            persona = "🔴 Struggling - Intensive Support"

    persona_mapping[cluster_id] = persona

df["persona_label"] = df["cluster"].map(persona_mapping)

# ---------------------------------------------------------
# 6. ENHANCED RISK DETECTION (GRANULAR & SPECIFIC)
# ---------------------------------------------------------
def detect_comprehensive_risks(row):
    risks = []
    risk_scores = []

    # ACADEMIC RISKS (Most specific)
    if row["avg_grade"] < 8:
        risks.append("🔴 Critical - Failing grades")
        risk_scores.append(0.95)
    elif row["avg_grade"] < 10 and row["grade_trend"] < 0:
        risks.append("🔴 Severe - Declining + Below 10")
        risk_scores.append(0.85)
    elif row["avg_grade"] < 10:
        risks.append("🟠 Moderate - Consistently low (<10)")
        risk_scores.append(0.70)
    elif row["grade_trend"] < -3:
        risks.append("🟠 Moderate - Sharp grade decline (>3pts)")
        risk_scores.append(0.75)
    elif row["grade_trend"] < -1 and row["absences"] > 5:
        risks.append("🟡 Low - Declining + Increased absences")
        risk_scores.append(0.55)
    elif row["avg_grade"] < 12 and row["grade_variance"] > 4:
        risks.append("🟡 Low - Inconsistent performance")
        risk_scores.append(0.50)

    # ATTENDANCE & BEHAVIOR
    if row["absences"] > 25:
        risks.append("🔴 Critical - Chronic absenteeism (>25)")
        risk_scores.append(0.90)
    elif row["absences"] > 15:
        risks.append("🟠 Moderate - High absence rate (>15)")
        risk_scores.append(0.65)
    elif row["absences"] > 8:
        risks.append("🟡 Low - Notable absences (>8)")
        risk_scores.append(0.45)

    # STUDY EFFORT & DISCIPLINE
    if row["studytime"] <= 1 and row["avg_grade"] < 10:
        risks.append("🔴 Critical - No study + Failing")
        risk_scores.append(0.88)
    elif row["studytime"] <= 2 and row["avg_grade"] < 12:
        risks.append("🟠 Moderate - Minimal study (<2hrs)")
        risk_scores.append(0.68)
    elif row["study_discipline"] < 0.2:
        risks.append("🟡 Low - Poor study habits")
        risk_scores.append(0.48)

    # ENGAGEMENT & MOTIVATION
    if row["engagement"] < 0.2 and row["higher"] == "no":
        risks.append("🔴 Critical - No engagement + No higher ed plans")
        risk_scores.append(0.82)
    elif row["engagement"] < 0.4 and row["schoolsup"] == "no":
        risks.append("🟠 Moderate - Disengaged + No school support")
        risk_scores.append(0.70)
    elif row["engagement"] < 0.3:
        risks.append("🟡 Low - Low engagement (<0.3)")
        risk_scores.append(0.50)
    elif row["higher"] == "no" and row["avg_grade"] < 10:
        risks.append("🟡 Low - Low grades + No higher ed aspiration")
        risk_scores.append(0.52)

    # FAMILY & SOCIOECONOMIC
    if row["low_sES"] > 0.7 and row["family_support"] < 0.3:
        risks.append("🟠 Moderate - High SES disadvantage + Weak family")
        risk_scores.append(0.72)
    elif row["family_support"] < 0.2:
        risks.append("🟡 Low - Minimal family support")
        risk_scores.append(0.48)
    elif row["famsup"] == "no" and row["avg_grade"] < 11:
        risks.append("🟡 Low - Missing school-approved support")
        risk_scores.append(0.50)

    # HEALTH & LIFESTYLE
    if row["lifestyle_risk"] > 0.7 and row["health_score"] > 0.7:
        risks.append("🟠 Moderate - High substance use + Health issues")
        risk_scores.append(0.74)
    elif row["lifestyle_risk"] > 0.7:
        risks.append("🟠 Moderate - High substance use risk")
        risk_scores.append(0.65)
    elif row["health_score"] > 0.7:
        risks.append("🟡 Low - Notable health concerns")
        risk_scores.append(0.48)
    elif row["Walc"] >= 4 and row["Dalc"] >= 3:
        risks.append("🟡 Low - Elevated weekend & weekday drinking")
        risk_scores.append(0.52)

    # SOCIAL FACTORS
    if row["goout"] >= 5 and row["study_discipline"] < 0.3:
        risks.append("🟡 Low - High social activity + Low discipline")
        risk_scores.append(0.50)

    # If no specific risks detected
    if not risks:
        risks.append("✓ No major risks detected")
        risk_scores.append(0)

    return risks, np.mean(risk_scores) if risk_scores else 0

df[["risk_flags", "risk_score"]] = df.apply(
    lambda row: pd.Series(detect_comprehensive_risks(row)), axis=1
)

# ---------------------------------------------------------
# 7. INTELLIGENT TEACHING STRATEGIES
# ---------------------------------------------------------
def generate_teaching_strategy(row):
    persona = row["persona_label"]
    risk_level = row["risk_score"]

    base_strategies = {
        "🌟 High Achiever - Consistent Excellence": [
            "Provide advanced enrichment & independent projects",
            "Assign peer tutoring/mentoring roles",
            "Prepare for competitive higher education",
            "Encourage leadership & research opportunities"
        ],
        "💎 High Potential - Needs Discipline": [
            "Teach structured time management systems",
            "Set firm deadlines with accountability",
            "One-on-one mentoring for focus improvement",
            "Monitor lifestyle/wellness factors"
        ],
        "✅ Solid Performer - On Track": [
            "Continue current support approach",
            "Gradually increase challenge level",
            "Encourage diverse academic interests",
            "Build towards career planning"
        ],
        "⚠️ Good Grades - Attendance Issues": [
            "Schedule parent-teacher meeting on attendance",
            "Identify barriers to school attendance",
            "Create attendance incentive plan",
            "Connect with school counselor"
        ],
        "📈 Above Average - Stable": [
            "Provide consistent encouragement",
            "Explore advanced topic areas",
            "Build student confidence systematically",
            "Set stretch goals for growth"
        ],
        "🔄 Developing Learner - Engaged": [
            "Use scaffolded, progressive assignments",
            "Provide regular formative feedback",
            "Build on existing motivation",
            "Create achievable milestone goals"
        ],
        "💪 Struggling but Trying": [
            "Validate effort & incremental progress",
            "Conduct diagnostic skill assessments",
            "Provide targeted tutoring in weak areas",
            "Use peer support & study groups"
        ],
        "⚠️ Low Engagement - Support Needed": [
            "Investigate root causes of disengagement",
            "Connect learning to student interests",
            "Increase teacher-student interaction",
            "Involve family in re-engagement plan"
        ],
        "🔴 At-Risk - High Absenteeism": [
            "Urgent: Attendance intervention meeting",
            "Coordinate with school counselor",
            "Explore transportation/family barriers",
            "Create structured daily check-in system"
        ],
        "🔴 At-Risk - Disengaged": [
            "Priority parent-teacher conference",
            "Assign academic mentor/buddy",
            "Recommend counseling services",
            "Daily engagement monitoring"
        ],
        "🔴 At-Risk - Lifestyle Issues": [
            "Confidential discussion on health concerns",
            "Refer to school counselor/social worker",
            "Create supportive classroom environment",
            "Weekly progress check-ins"
        ],
        "🔴 Struggling - Intensive Support": [
            "Implement intensive tutoring program",
            "Schedule daily learning check-ins",
            "Coordinate multi-agency support",
            "Create realistic short-term goals"
        ]
    }

    strategies = base_strategies.get(persona, [])

    # Add urgency for very high risk
    if risk_level > 0.75:
        strategies.insert(0, "⚠️ IMMEDIATE PRIORITY INTERVENTION")

    return strategies

df["teaching_strategy"] = df.apply(generate_teaching_strategy, axis=1)

# ---------------------------------------------------------
# 8. COMPREHENSIVE MODEL EVALUATION
# ---------------------------------------------------------
print("\n" + "="*60)
print("MODEL EVALUATION METRICS")
print("="*60 + "\n")

# Clustering Quality
sil_score = silhouette_score(X, df["cluster"])
db_score = davies_bouldin_score(X, df["cluster"])
ch_score = calinski_harabasz_score(X, df["cluster"])

print("Clustering Quality:")
print(f"  Silhouette Score (↑ better, range: -1 to 1): {sil_score:.3f}")
print(f"  Davies-Bouldin Index (↓ better): {db_score:.3f}")
print(f"  Calinski-Harabasz Score (↑ better): {ch_score:.1f}\n")

# Cluster Distribution
print("Student Distribution by Persona:")
persona_dist = df["persona_label"].value_counts().sort_values(ascending=False)
for persona, count in persona_dist.items():
    pct = (count / len(df)) * 100
    print(f"  {persona}: {count} ({pct:.1f}%)")

print("\n" + "-"*60)
print("Academic Performance by Persona:")
print("-"*60)
performance = df.groupby("persona_label").agg({
    "avg_grade": ["mean", "std"],
    "grade_trend": "mean",
    "absences": "mean"
}).round(2)
print(performance)

print("\n" + "-"*60)
print("Risk Assessment Summary:")
print("-"*60)
print(f"High Risk Students (score > 0.7): {len(df[df['risk_score'] > 0.7])}")
print(f"Medium Risk Students (0.4-0.7): {len(df[(df['risk_score'] >= 0.4) & (df['risk_score'] <= 0.7)])}")
print(f"Low Risk Students (< 0.4): {len(df[df['risk_score'] < 0.4])}\n")

# ---------------------------------------------------------
# 9. TEACHER DASHBOARD OUTPUT
# ---------------------------------------------------------
teacher_dashboard = df[[
    "persona_label",
    "avg_grade",
    "grade_trend",
    "study_discipline",
    "engagement",
    "absences",
    "risk_score",
    "risk_flags",
    "teaching_strategy"
]].copy()

teacher_dashboard = teacher_dashboard.sort_values("risk_score", ascending=False)

print("\n" + "="*60)
print("TEACHER DASHBOARD (PRIORITIZED BY RISK)")
print("="*60 + "\n")
print(teacher_dashboard.head(15).to_string())

print("\n[Showing top 15 at-risk students. Full report available for export.]")

# ---------------------------------------------------------
# 10. EXPORT OPTIONS
# ---------------------------------------------------------
# Uncomment to save detailed reports
# teacher_dashboard.to_csv("teacher_report_detailed.csv", index=False)
# df.to_csv("student_analysis_full.csv", index=False)
print("\n✓ Analysis complete. Ready for export or further analysis.")



# =========================================================
# Student Learning Analysis Dashboard - CustomTkinter UI
# =========================================================

import customtkinter as ctk
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

# Set appearance mode
ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("blue")

# Global variables
df_global = None
analysis_complete = False

class StudentAnalysisDashboard(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.title("Student Learning Pattern Analysis Dashboard")
        self.geometry("1400x900")
        self.resizable(True, True)
        
        # Configure grid
        self.grid_rowconfigure(0, weight=1)
        self.grid_columnconfigure(0, weight=1)
        
        # Create main container
        self.main_container = ctk.CTkFrame(self)
        self.main_container.grid(row=0, column=0, sticky="nsew", padx=10, pady=10)
        self.main_container.grid_rowconfigure(1, weight=1)
        self.main_container.grid_columnconfigure(0, weight=1)
        
        # Create header
        self.create_header()
        
        # Create content frame
        self.content_frame = ctk.CTkFrame(self.main_container)
        self.content_frame.grid(row=1, column=0, sticky="nsew", padx=5, pady=10)
        self.content_frame.grid_rowconfigure(0, weight=1)
        self.content_frame.grid_columnconfigure(0, weight=1)
        self.content_frame.grid_columnconfigure(1, weight=2)
        
        # Create left panel (Student selector)
        self.create_left_panel()
        
        # Create right panel (Student details)
        self.create_right_panel()
        
        # Bind window close event
        self.protocol("WM_DELETE_WINDOW", self.on_closing)
    
    def create_header(self):
        header_frame = ctk.CTkFrame(self.main_container, fg_color="transparent")
        header_frame.grid(row=0, column=0, sticky="ew", padx=5, pady=5)
        header_frame.grid_columnconfigure(1, weight=1)
        
        # Title
        title = ctk.CTkLabel(
            header_frame,
            text="📊 Student Learning Pattern Analysis Dashboard",
            font=("Arial", 22, "bold")
        )
        title.grid(row=0, column=0, sticky="w")
        
        # Button frame
        button_frame = ctk.CTkFrame(header_frame, fg_color="transparent")
        button_frame.grid(row=0, column=2, sticky="e")
        
        load_btn = ctk.CTkButton(
            button_frame,
            text="📁 Load CSV",
            command=self.load_csv,
            width=120
        )
        load_btn.pack(side="left", padx=5)
        
        analyze_btn = ctk.CTkButton(
            button_frame,
            text="🔍 Analyze",
            command=self.run_analysis,
            width=120
        )
        analyze_btn.pack(side="left", padx=5)
    
    def create_left_panel(self):
        left_frame = ctk.CTkFrame(self.content_frame)
        left_frame.grid(row=0, column=0, sticky="nsew", padx=5, pady=5)
        left_frame.grid_rowconfigure(3, weight=1)
        left_frame.grid_columnconfigure(0, weight=1)
        
        # Student selector label
        label = ctk.CTkLabel(left_frame, text="Select Student", font=("Arial", 14, "bold"))
        label.grid(row=0, column=0, sticky="w", padx=10, pady=10)
        
        # Search frame
        search_frame = ctk.CTkFrame(left_frame, fg_color="transparent")
        search_frame.grid(row=1, column=0, sticky="ew", padx=10, pady=5)
        search_frame.grid_columnconfigure(0, weight=1)
        
        ctk.CTkLabel(search_frame, text="Search:", font=("Arial", 10)).pack(side="left", padx=5)
        
        self.search_var = ctk.StringVar()
        self.search_var.trace("w", self.update_student_list)
        
        search_entry = ctk.CTkEntry(
            search_frame,
            textvariable=self.search_var,
            placeholder_text="By name or index..."
        )
        search_entry.pack(side="left", fill="x", expand=True, padx=5)
        
        # Student listbox with scrollbar
        list_frame = ctk.CTkFrame(left_frame)
        list_frame.grid(row=2, column=0, sticky="nsew", padx=10, pady=5)
        list_frame.grid_rowconfigure(0, weight=1)
        list_frame.grid_columnconfigure(0, weight=1)
        
        self.student_listbox = tk.Listbox(
            list_frame,
            font=("Arial", 10),
            bg="#212121",
            fg="white",
            selectmode="SINGLE",
            highlightcolor="#0080FF"
        )
        self.student_listbox.grid(row=0, column=0, sticky="nsew")
        self.student_listbox.bind("<<ListboxSelect>>", self.on_student_select)
        
        scrollbar = ttk.Scrollbar(list_frame, orient="vertical", command=self.student_listbox.yview)
        scrollbar.grid(row=0, column=1, sticky="ns")
        self.student_listbox.config(yscrollcommand=scrollbar.set)
        
        # Filter by persona
        filter_label = ctk.CTkLabel(left_frame, text="Filter by Persona:", font=("Arial", 10, "bold"))
        filter_label.grid(row=3, column=0, sticky="w", padx=10, pady=(15, 5))
        
        self.persona_filter = ctk.CTkComboBox(
            left_frame,
            values=["All Personas"],
            state="disabled",
            command=self.filter_by_persona
        )
        self.persona_filter.grid(row=4, column=0, sticky="ew", padx=10, pady=5)
    
    def create_right_panel(self):
        right_frame = ctk.CTkFrame(self.content_frame)
        right_frame.grid(row=0, column=1, sticky="nsew", padx=5, pady=5)
        right_frame.grid_rowconfigure(1, weight=1)
        right_frame.grid_columnconfigure(0, weight=1)
        
        # Student info title
        info_title = ctk.CTkLabel(
            right_frame,
            text="Student Details & Analysis",
            font=("Arial", 14, "bold")
        )
        info_title.grid(row=0, column=0, sticky="w", padx=10, pady=10)
        
        # Scrollable frame for details
        scrollable_frame = ctk.CTkScrollableFrame(right_frame)
        scrollable_frame.grid(row=1, column=0, sticky="nsew", padx=10, pady=10)
        scrollable_frame.grid_columnconfigure(0, weight=1)
        
        # Personal Info Section
        self.create_section(scrollable_frame, "👤 Personal Information")
        
        self.info_labels = {}
        personal_fields = ["Index", "Name", "Age", "School", "Address"]
        for field in personal_fields:
            self.info_labels[field] = self.create_info_field(scrollable_frame, field)
        
        # Academic Section
        self.create_section(scrollable_frame, "📚 Academic Performance")
        
        academic_fields = ["G1", "G2", "G3", "Average Grade", "Grade Trend", "Grade Stability"]
        for field in academic_fields:
            self.info_labels[field] = self.create_info_field(scrollable_frame, field)
        
        # Behavioral Section
        self.create_section(scrollable_frame, "📊 Behavioral & Engagement")
        
        behavioral_fields = ["Study Discipline", "Engagement", "Absences", "Travel Time"]
        for field in behavioral_fields:
            self.info_labels[field] = self.create_info_field(scrollable_frame, field)
        
        # Family & Support Section
        self.create_section(scrollable_frame, "👨‍👩‍👧 Family & Support")
        
        family_fields = ["Family Support", "Parent Education", "Family Relations", "School Support"]
        for field in family_fields:
            self.info_labels[field] = self.create_info_field(scrollable_frame, field)
        
        # Lifestyle Section
        self.create_section(scrollable_frame, "🎯 Lifestyle & Health")
        
        lifestyle_fields = ["Lifestyle Risk", "Health Status", "Weekend Alcohol", "Weekday Alcohol"]
        for field in lifestyle_fields:
            self.info_labels[field] = self.create_info_field(scrollable_frame, field)
        
        # Analysis Results Section
        self.create_section(scrollable_frame, "🔍 Analysis Results")
        
        analysis_fields = ["Learning Persona", "Risk Score", "Risk Flags", "Teaching Strategies"]
        for field in analysis_fields:
            self.info_labels[field] = self.create_info_field(scrollable_frame, field)
    
    def create_section(self, parent, title):
        section = ctk.CTkFrame(parent, fg_color="#1a1a1a", border_width=2, border_color="#0080FF")
        section.pack(fill="x", pady=(15, 5), padx=0)
        section.grid_columnconfigure(0, weight=1)
        
        section_title = ctk.CTkLabel(
            section,
            text=title,
            font=("Arial", 12, "bold"),
            text_color="#0080FF"
        )
        section_title.grid(row=0, column=0, sticky="w", padx=15, pady=10)
    
    def create_info_field(self, parent, field_name):
        field_frame = ctk.CTkFrame(parent, fg_color="transparent")
        field_frame.pack(fill="x", padx=10, pady=5)
        field_frame.grid_columnconfigure(1, weight=1)
        
        label = ctk.CTkLabel(
            field_frame,
            text=f"{field_name}:",
            font=("Arial", 10, "bold"),
            width=180,
            justify="left"
        )
        label.grid(row=0, column=0, sticky="w", padx=5)
        
        value_label = ctk.CTkLabel(
            field_frame,
            text="—",
            font=("Arial", 10),
            text_color="#90EE90",
            justify="left",
            wraplength=400
        )
        value_label.grid(row=0, column=1, sticky="w", padx=10)
        
        return value_label
    
    def load_csv(self):
        global df_global
        file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
        if not file_path:
            return
        
        try:
            df_global = pd.read_csv(file_path)
            
            # Standardize column names (case-insensitive)
            df_global.columns = df_global.columns.str.lower().str.strip()
            
            # Print available columns for debugging
            print("Available columns:", df_global.columns.tolist())
            
            # Ensure critical columns exist
            if 'g1' not in df_global.columns or 'g2' not in df_global.columns or 'g3' not in df_global.columns:
                messagebox.showerror("Error", "CSV must contain G1, G2, G3 columns")
                return
            
            df_global = df_global.dropna(subset=['g1', 'g2', 'g3'])
            df_global = df_global[
                (df_global['g1'] >= 0) & (df_global['g1'] <= 20) &
                (df_global['g2'] >= 0) & (df_global['g2'] <= 20) &
                (df_global['g3'] >= 0) & (df_global['g3'] <= 20)
            ].reset_index(drop=True)
            
            messagebox.showinfo("Success", f"Loaded {len(df_global)} students successfully!")
            self.update_student_list()
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load CSV: {str(e)}")
    
    def run_analysis(self):
        global df_global, analysis_complete
        
        if df_global is None or len(df_global) == 0:
            messagebox.showwarning("Warning", "Please load a CSV file first!")
            return
        
        try:
            # Standardize column names
            df_global.columns = df_global.columns.str.lower().str.strip()
            
            print("Starting analysis...")
            print(f"DataFrame shape: {df_global.shape}")
            print(f"Columns: {df_global.columns.tolist()}")
            
            # Ensure G1, G2, G3 exist
            if 'g1' not in df_global.columns or 'g2' not in df_global.columns or 'g3' not in df_global.columns:
                messagebox.showerror("Error", "CSV must have G1, G2, G3 columns (grade columns)")
                return
            
            # Feature Engineering with safe column access
            df_global["avg_grade"] = df_global[["g1", "g2", "g3"]].mean(axis=1)
            df_global["grade_variance"] = df_global[["g1", "g2", "g3"]].var(axis=1)
            df_global["grade_trend"] = df_global["g3"] - df_global["g1"]
            df_global["grade_stability"] = 1 / (1 + df_global["grade_variance"].fillna(0.1))
            
            print(f"✓ Calculated: avg_grade, grade_variance, grade_trend, grade_stability")
            
            # Handle optional columns with defaults
            studytime = df_global.get("studytime", pd.Series([2]*len(df_global))) if "studytime" in df_global.columns else pd.Series([2]*len(df_global))
            traveltime = df_global.get("traveltime", pd.Series([1]*len(df_global))) if "traveltime" in df_global.columns else pd.Series([1]*len(df_global))
            absences = df_global.get("absences", pd.Series([0]*len(df_global))) if "absences" in df_global.columns else pd.Series([0]*len(df_global))
            
            df_global["study_discipline"] = (
                (studytime / 4) * 0.35 +
                (1 - traveltime / 4) * 0.25 +
                (1 - absences / (absences.max() + 1)) * 0.40
            ).fillna(0.5)
            
            print(f"✓ Calculated: study_discipline")
            
            # Engagement factors
            activities = df_global["activities"].map({"yes": 1, "no": 0}).fillna(0) if "activities" in df_global.columns else pd.Series([0]*len(df_global))
            higher = df_global["higher"].map({"yes": 1, "no": 0}).fillna(0) if "higher" in df_global.columns else pd.Series([0]*len(df_global))
            schoolsup = df_global["schoolsup"].map({"yes": 1, "no": 0}).fillna(0) if "schoolsup" in df_global.columns else pd.Series([0]*len(df_global))
            internet = df_global["internet"].map({"yes": 1, "no": 0}).fillna(0) if "internet" in df_global.columns else pd.Series([0]*len(df_global))
            paid = df_global["paid"].map({"yes": 1, "no": 0}).fillna(0) if "paid" in df_global.columns else pd.Series([0]*len(df_global))
            
            df_global["engagement"] = (
                activities * 0.25 + higher * 0.20 + schoolsup * 0.20 + internet * 0.20 + paid * 0.15
            ).fillna(0.3)
            
            print(f"✓ Calculated: engagement")
            
            # Family support
            medu = df_global["medu"].astype(float) if "medu" in df_global.columns else pd.Series([2.0]*len(df_global))
            fedu = df_global["fedu"].astype(float) if "fedu" in df_global.columns else pd.Series([2.0]*len(df_global))
            famsup = df_global["famsup"].map({"yes": 1, "no": 0}).fillna(0) if "famsup" in df_global.columns else pd.Series([0]*len(df_global))
            famrel = df_global["famrel"].astype(float) if "famrel" in df_global.columns else pd.Series([3.0]*len(df_global))
            
            df_global["parent_education"] = ((medu + fedu) / 8).fillna(0.3)
            df_global["family_support"] = (
                (df_global["parent_education"] * 0.35) +
                (famsup * 0.30) +
                ((famrel / 5) * 0.35)
            ).fillna(0.4)
            
            print(f"✓ Calculated: parent_education, family_support")
            
            # Socioeconomic status
            pstatus = df_global["pstatus"].map({"t": 0, "a": 1}).fillna(0) if "pstatus" in df_global.columns else pd.Series([0.0]*len(df_global))
            df_global["low_ses"] = (
                (pstatus * 0.5) +
                (1 - medu / 4) * 0.25 +
                (1 - fedu / 4) * 0.25
            ).fillna(0.3)
            
            print(f"✓ Calculated: low_ses")
            
            # Lifestyle risk
            goout = df_global["goout"].astype(float) if "goout" in df_global.columns else pd.Series([2.0]*len(df_global))
            dalc = df_global["dalc"].astype(float) if "dalc" in df_global.columns else pd.Series([1.0]*len(df_global))
            walc = df_global["walc"].astype(float) if "walc" in df_global.columns else pd.Series([1.0]*len(df_global))
            
            df_global["lifestyle_risk"] = (
                (goout / 5) * 0.30 +
                (dalc / 5) * 0.35 +
                (walc / 5) * 0.35
            ).fillna(0.2)
            
            print(f"✓ Calculated: lifestyle_risk")
            
            # Health score
            health = df_global["health"].astype(float) if "health" in df_global.columns else pd.Series([3.0]*len(df_global))
            df_global["health_score"] = (1 - (health / 5)).fillna(0.4)
            
            print(f"✓ Calculated: health_score")
            
            # Clustering
            feature_list = [
                "avg_grade", "grade_stability", "grade_trend", "study_discipline",
                "engagement", "family_support", "lifestyle_risk", "health_score", "low_ses"
            ]
            
            scaler = StandardScaler()
            X = scaler.fit_transform(df_global[feature_list].fillna(0))
            
            kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
            df_global["cluster"] = kmeans.fit_predict(X)
            
            print(f"✓ Clustering complete")
            
            # Persona mapping
            persona_mapping = {}
            for cluster_id in range(5):
                cluster_data = df_global[df_global["cluster"] == cluster_id]
                avg_grade = cluster_data["avg_grade"].mean()
                avg_engagement = cluster_data["engagement"].mean()
                avg_discipline = cluster_data["study_discipline"].mean()
                avg_absence = cluster_data["absences"].mean() if "absences" in df_global.columns else 5
                avg_risk = cluster_data["lifestyle_risk"].mean()
                
                if avg_grade >= 15:
                    if avg_discipline >= 0.6:
                        persona = "🌟 High Achiever"
                    else:
                        persona = "💎 High Potential"
                elif avg_grade >= 12:
                    if avg_discipline >= 0.5 and avg_engagement >= 0.5:
                        persona = "✅ Solid Performer"
                    elif avg_absence >= 15:
                        persona = "⚠️ Attendance Issues"
                    else:
                        persona = "📈 Above Average"
                elif avg_grade >= 10:
                    if avg_engagement >= 0.5:
                        persona = "🔄 Developing Learner"
                    elif avg_discipline >= 0.5:
                        persona = "💪 Struggling but Trying"
                    else:
                        persona = "⚠️ Low Engagement"
                else:
                    if avg_absence >= 20:
                        persona = "🔴 At-Risk (Absenteeism)"
                    elif avg_engagement < 0.3:
                        persona = "🔴 At-Risk (Disengaged)"
                    elif avg_risk > 0.6:
                        persona = "🔴 At-Risk (Lifestyle)"
                    else:
                        persona = "🔴 Struggling"
                
                persona_mapping[cluster_id] = persona
            
            df_global["persona_label"] = df_global["cluster"].map(persona_mapping)
            
            print(f"✓ Personas assigned")
            
            # Risk detection
            def detect_risks(row):
                risks = []
                risk_scores = []
                
                if row["avg_grade"] < 8:
                    risks.append("🔴 Critical - Failing")
                    risk_scores.append(0.95)
                elif row["avg_grade"] < 10 and row["grade_trend"] < 0:
                    risks.append("🔴 Severe - Declining")
                    risk_scores.append(0.85)
                elif row["avg_grade"] < 10:
                    risks.append("🟠 Moderate - Low grades")
                    risk_scores.append(0.70)
                
                if row.get("absences", 0) > 25:
                    risks.append("🔴 Critical - Absenteeism")
                    risk_scores.append(0.90)
                elif row.get("absences", 0) > 15:
                    risks.append("🟠 Moderate - High absences")
                    risk_scores.append(0.65)
                
                if row.get("studytime", 2) <= 1 and row["avg_grade"] < 10:
                    risks.append("🔴 Critical - No study")
                    risk_scores.append(0.88)
                
                if not risks:
                    risks.append("✓ No major risks")
                    risk_scores.append(0)
                
                return risks, np.mean(risk_scores) if risk_scores else 0
            
            df_global[["risk_flags", "risk_score"]] = df_global.apply(
                lambda row: pd.Series(detect_risks(row)), axis=1
            )
            
            print(f"✓ Risk detection complete")
            
            # Teaching strategies
            def teaching_strategy(persona):
                strategies = {
                    "🌟 High Achiever": ["Advanced enrichment", "Peer mentoring", "Leadership roles"],
                    "💎 High Potential": ["Time management", "Structured deadlines", "Mentoring"],
                    "✅ Solid Performer": ["Continue support", "Increase challenges", "Career planning"],
                    "⚠️ Attendance Issues": ["Attendance intervention", "Parent meeting", "Attendance plan"],
                    "📈 Above Average": ["Encouragement", "Advanced topics", "Stretch goals"],
                    "🔄 Developing Learner": ["Scaffolded assignments", "Regular feedback", "Milestones"],
                    "💪 Struggling but Trying": ["Validate effort", "Skill assessments", "Tutoring"],
                    "⚠️ Low Engagement": ["Investigate causes", "Connect to interests", "Family involvement"],
                    "🔴 At-Risk (Absenteeism)": ["Attendance meeting", "Counselor coordination", "Daily check-ins"],
                    "🔴 At-Risk (Disengaged)": ["Parent conference", "Academic mentor", "Daily monitoring"],
                    "🔴 At-Risk (Lifestyle)": ["Health discussion", "Counselor referral", "Weekly check-ins"],
                    "🔴 Struggling": ["Intensive tutoring", "Daily check-ins", "Multi-agency support"]
                }
                return strategies.get(persona, ["Provide general support"])
            
            df_global["teaching_strategy"] = df_global["persona_label"].apply(teaching_strategy)
            
            print(f"✓ Teaching strategies assigned")
            
            analysis_complete = True
            print("✓ ANALYSIS COMPLETE!")
            messagebox.showinfo("Success", "Analysis completed successfully!\n\nClick on a student to see results.")
            self.update_student_list()
            self.update_persona_filter()
            
        except Exception as e:
            print(f"ERROR: {str(e)}")
            import traceback
            traceback.print_exc()
            messagebox.showerror("Error", f"Analysis failed: {str(e)}")
    
    def update_student_list(self, *args):
        global df_global
        
        if df_global is None or len(df_global) == 0:
            self.student_listbox.delete(0, tk.END)
            return
        
        search_text = self.search_var.get().lower()
        self.student_listbox.delete(0, tk.END)
        
        filtered_df = df_global
        
        # Filter by search
        if search_text:
            mask = (
                df_global.index.astype(str).str.contains(search_text) |
                df_global.get('schoolsex', pd.Series([False]*len(df_global))).astype(str).str.contains(search_text, case=False)
            )
            filtered_df = df_global[mask]
        
        # Filter by persona if applicable
        if analysis_complete and self.persona_filter.get() != "All Personas":
            filtered_df = filtered_df[filtered_df["persona_label"] == self.persona_filter.get()]
        
        for idx, row in filtered_df.iterrows():
            display_name = f"[{idx}] Student {idx}"
            self.student_listbox.insert(tk.END, display_name)
            self.student_listbox.itemconfig(tk.END, {"bg": "#1a1a1a", "fg": "white"})
    
    def update_persona_filter(self):
        global df_global, analysis_complete
        
        if analysis_complete and df_global is not None:
            personas = ["All Personas"] + sorted(df_global["persona_label"].unique().tolist())
            self.persona_filter.configure(values=personas, state="normal")
            self.persona_filter.set("All Personas")
    
    def filter_by_persona(self, value):
        self.update_student_list()
    
    def on_student_select(self, event):
        global df_global, analysis_complete
        
        if df_global is None:
            return
        
        selection = self.student_listbox.curselection()
        if not selection:
            return
        
        selected_text = self.student_listbox.get(selection[0])
        student_idx = int(selected_text.split("]")[0][1:])
        
        student = df_global.iloc[student_idx]
        
        # Helper function to safely get values
        def get_value(col, default="N/A", decimals=None):
            try:
                val = student.get(col, default)
                if val is None or (isinstance(val, float) and pd.isna(val)):
                    return default
                if decimals is not None and isinstance(val, (int, float)):
                    return f"{float(val):.{decimals}f}"
                return str(val)
            except:
                return default
        
        # Update personal info
        self.update_label("Index", str(student_idx))
        self.update_label("Name", f"Student {student_idx}")
        self.update_label("Age", get_value("age"))
        self.update_label("School", get_value("school"))
        self.update_label("Address", get_value("address"))
        
        # Update academic info - these are CALCULATED fields
        if analysis_complete:
            self.update_label("G1", get_value("g1", decimals=2))
            self.update_label("G2", get_value("g2", decimals=2))
            self.update_label("G3", get_value("g3", decimals=2))
            self.update_label("Average Grade", get_value("avg_grade", decimals=2))
            self.update_label("Grade Trend", get_value("grade_trend", decimals=2))
            self.update_label("Grade Stability", get_value("grade_stability", decimals=2))
        else:
            for field in ["G1", "G2", "G3", "Average Grade", "Grade Trend", "Grade Stability"]:
                self.update_label(field, "Run Analysis First")
        
        # Update behavioral info
        self.update_label("Study Discipline", get_value("study_discipline", decimals=2))
        self.update_label("Engagement", get_value("engagement", decimals=2))
        self.update_label("Absences", get_value("absences"))
        self.update_label("Travel Time", get_value("traveltime"))
        
        # Update family info
        self.update_label("Family Support", get_value("family_support", decimals=2))
        self.update_label("Parent Education", get_value("parent_education", decimals=2))
        self.update_label("Family Relations", get_value("famrel"))
        self.update_label("School Support", get_value("schoolsup"))
        
        # Update lifestyle info
        self.update_label("Lifestyle Risk", get_value("lifestyle_risk", decimals=2))
        self.update_label("Health Status", get_value("health"))
        self.update_label("Weekend Alcohol", get_value("walc"))
        self.update_label("Weekday Alcohol", get_value("dalc"))
        
        # Update analysis results - only if analysis is complete
        if analysis_complete:
            self.update_label("Learning Persona", get_value("persona_label"))
            self.update_label("Risk Score", get_value("risk_score", decimals=2))
            
            risk_flags = student.get('risk_flags', [])
            risk_text = "\n".join(risk_flags) if isinstance(risk_flags, list) else str(risk_flags)
            self.info_labels["Risk Flags"].configure(text=risk_text if risk_text else "N/A", text_color="#FFB6C6")
            
            strategies = student.get('teaching_strategy', [])
            strategy_text = "\n".join(f"• {s}" for s in strategies) if isinstance(strategies, list) else str(strategies)
            self.info_labels["Teaching Strategies"].configure(text=strategy_text if strategy_text else "N/A", text_color="#FFD700")
        else:
            self.update_label("Learning Persona", "Run Analysis First")
            self.update_label("Risk Score", "Run Analysis First")
            self.info_labels["Risk Flags"].configure(text="Run Analysis First", text_color="#FFB6C6")
            self.info_labels["Teaching Strategies"].configure(text="Run Analysis First", text_color="#FFD700")
    
    def update_label(self, field_name, value):
        if field_name in self.info_labels:
            self.info_labels[field_name].configure(text=str(value))
    
    def on_closing(self):
        self.destroy()

if __name__ == "__main__":
    app = StudentAnalysisDashboard()
    app.mainloop()

Dataset shape: (6395, 34)
Missing values:
school           0
sex              0
age              0
address          0
famsize          0
Pstatus          0
Medu             0
Fedu             0
Mjob             0
Fjob             0
reason           0
guardian         0
traveltime       0
studytime        0
failures         0
schoolsup        0
famsup           0
paid             0
activities       0
nursery          0
higher           0
internet         0
romantic         0
famrel           0
freetime         0
goout            0
Dalc             0
Walc             0
health           0
absences         0
G1               0
G2               0
G3               0
part_time_job    0
dtype: int64

Optimal clusters: 3 (Silhouette: 0.100)


MODEL EVALUATION METRICS

Clustering Quality:
  Silhouette Score (↑ better, range: -1 to 1): 0.100
  Davies-Bouldin Index (↓ better): 2.668
  Calinski-Harabasz Score (↑ better): 641.4

Student Distribution by Persona:
  📈 Above Average - Stable: 2504 (39.2